# Python и интернет. Модуль requests

**План**:

1. Запросы
2. Requests
4. bs4
5. Задание на семинар

## Как выкачать интернет
Современный Интернет предоставляет лингвистам большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть несколько библиотек **urllib** и **requests**

## Requests
Requests является уже стандартной библиотекой. Поэтому, после установки python она уже у вас есть.
Можно почитать еще [тут](https://realpython.com/python-requests/)


Допустим, мы хотим скачать главную страницу Хабрахабра.

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер **запрос** ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать **запрос** на сервер так же, как это делает браузер:

In [3]:
import requests

In [20]:
# import ssl
# gcontext = ssl.SSLContext() 
# _create_unverified_https_context = ssl._create_unverified_context
# ssl._create_default_https_context = _create_unverified_https_context

Давайте попробуем отправить простой запрос на сайт хабра.

In [4]:
response = requests.get("https://habr.com/ru/")

В response теперь лежит отет сервера. Это не прост html-код страницы, а еще дополнительная информация. Если мы просто выведем этот response, нам покажется только код (200 - все ок).

In [5]:
response

<Response [200]>

In [6]:
response.status_code

200

А вот в атрибуте text уже лежит html-код

In [10]:
len(response.text) # ответ может быть достаточно большим

188866

In [18]:
print(response.text[:210])

<!DOCTYPE html>
<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">
<head >
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cov


Можно посмотреть, что еще можно достать из response.

Функция ```dir``` выдает список методов и параметров объекта.

In [19]:
[i for i in dir(response) if i.startswith("_")]

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next']

In [20]:
[i for i in dir(response) if not i.startswith("_")]

['apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

In [37]:
response.encoding # Кодировка 

'utf-8'

In [38]:
dict(response.headers) # Заголовки (техническая информация)

{'Server': 'QRATOR',
 'Date': 'Fri, 18 Nov 2022 14:04:40 GMT',
 'Content-Type': 'text/html; charset=utf-8',
 'Transfer-Encoding': 'chunked',
 'Connection': 'keep-alive',
 'Keep-Alive': 'timeout=15',
 'Vary': 'Accept-Encoding, Accept-Encoding',
 'X-DNS-Prefetch-Control': 'off',
 'X-Frame-Options': 'SAMEORIGIN',
 'X-Download-Options': 'noopen',
 'X-Content-Type-Options': 'nosniff',
 'X-XSS-Protection': '1; mode=block',
 'Cache-Control': 'no-store, no-cache, must-revalidate',
 'Pragma': 'no-cache',
 'Expires': '0',
 'ETag': 'W/"45d89-Pd6OFl4BNwupn5q6m2suz0nSKTQ"',
 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains',
 'X-Request-Id': '0ab3b0237b89ea4bdcf84e73da5b81e0',
 'X-Request-Geoip-Country-Code': 'RU',
 'X-Request-Detected-Device': 'desktop',
 'Content-Encoding': 'gzip'}

In [39]:
response.url # Адрес запроса

'https://habr.com/ru/all/'

In [40]:
print(response.text[:300])

<!DOCTYPE html>
<html lang="ru" data-vue-meta="%7B%22lang%22:%7B%22ssr%22:%22ru%22%7D%7D">
<head >
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width,initial-scale=1.0,viewport-fit=cover,maximum-scale=1,user-scalable=0">
  <meta name="referrer" content="unsafe-url">
  <titl


___
Перед нами ответ сервера в виде HTML документа. Не стоит его пугаться, как и JSON это всего лишь текстовый документ, но с определенной структурой.
**HTML это язык разметки сайтов.**

Давайте откроем сайт (пусть будет https://habr.com/ru/all/) и посмотрим как устроена его разметка:
1) Открываете сайт в Google Chrome
2) Кликаем правой кнопкой на окрывшейся странице, выбираем в открывшемся меняю - Посмотреть код
3) Смотрим какой блок в HTML коде страницы отвечает за тот или иной визуальный блок на сайте
___

Вся информация в HTML заключена в тэги. Например, между тегами `<title> </title>` лежит заголовок этой интернет-страницы.
Поэтому если вы хотите вытащить какую-то конкретную информацию из HTML документа, вам необходимо ориентороваться на эти теги.  

Для этого вы может написать свой парсер, который будет основан на регулярных выражениях, и с помощью него преобразовывать HTML в какой-то удобный python объект с которым удобно работать.

Или...

Воспользоваться готовыми решениями для парсинга HTML документов - BeautifulSoup (bs4) / pyquery / lxml

## BeautifulSoup

Код страницы парсится как иерархия тегов (как они есть в html коде, один вложен в другой) и можно искать элементы с помощью разных методов, использовать атрибуты и т.д.

In [ ]:
!pip install bs4

In [22]:
from bs4 import BeautifulSoup

In [88]:
type(BeautifulSoup)

type

In [90]:
# Парсим текст ответа с помошью BeautifulSoup
# На вход html документ и тип парсерса (есть разные парсеры под разные форматы данных)
soup = BeautifulSoup(response.text, 'html.parser')  

In [94]:
type(soup)

bs4.BeautifulSoup

Если вглядеться в HTML код страницы, мы увидим что многие теги имеют дополнительные параметры, например - class/lang/src/...
Их тоже можно учитывать при поиске нужной информации по html документу.  
Для поиска у нашего парсера `soup` есть метод `find`. В нем первым аргументом мы задаем тег, который ищем. Во втором аргументе словарем идут дополнительные параметры (class/lang/src/...).

In [91]:
# Этот запрос вернёт нам только первый заголовок. То есть первое вхождение такого тега в нашем html файле.
post = soup.find('h2', {'class': 'tm-article-snippet__title_h2'})

In [92]:
type(post)

bs4.element.Tag

In [93]:
print(post)

<h2 class="tm-article-snippet__title tm-article-snippet__title_h2" data-test-id="articleTitle"><a class="tm-article-snippet__title-link" data-article-link="" data-test-id="article-snippet-title-link" href="/ru/company/timeweb/blog/700224/"><span>Сборка мусора в неисправных JVM, проактивный подход</span></a></h2>


In [95]:
print(post.prettify())

<h2 class="tm-article-snippet__title tm-article-snippet__title_h2" data-test-id="articleTitle">
 <a class="tm-article-snippet__title-link" data-article-link="" data-test-id="article-snippet-title-link" href="/ru/company/timeweb/blog/700224/">
  <span>
   Сборка мусора в неисправных JVM, проактивный подход
  </span>
 </a>
</h2>



In [41]:
print(post.get_text()) # Название первой статьи с хабра

Сборка мусора в неисправных JVM, проактивный подход


___
Но мы хотим получить все заголовки постов! Метод find_all возвращает массив всех элементов с тегом указанным в скобках. По нему можно итерироваться.

In [44]:
all_articles = soup.find_all('h2', {'class': 'tm-article-snippet__title_h2'})
print(type(all_articles))
print(len(all_articles))

<class 'bs4.element.ResultSet'>
20


In [45]:
all_articles[0]

<h2 class="tm-article-snippet__title tm-article-snippet__title_h2" data-test-id="articleTitle"><a class="tm-article-snippet__title-link" data-article-link="" data-test-id="article-snippet-title-link" href="/ru/company/timeweb/blog/700224/"><span>Сборка мусора в неисправных JVM, проактивный подход</span></a></h2>

In [46]:
for post in all_articles[:3]:
    print(post.get_text())
    print(post.prettify())

    print('-- '*10)  # для красоты

Сборка мусора в неисправных JVM, проактивный подход
<h2 class="tm-article-snippet__title tm-article-snippet__title_h2" data-test-id="articleTitle">
 <a class="tm-article-snippet__title-link" data-article-link="" data-test-id="article-snippet-title-link" href="/ru/company/timeweb/blog/700224/">
  <span>
   Сборка мусора в неисправных JVM, проактивный подход
  </span>
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 
Трудности перевода в разработке: как делать интернациональные проекты и говорить с пользователями на одном языке
<h2 class="tm-article-snippet__title tm-article-snippet__title_h2" data-test-id="articleTitle">
 <a class="tm-article-snippet__title-link" data-article-link="" data-test-id="article-snippet-title-link" href="/ru/company/vk/blog/699852/">
  <span>
   Трудности перевода в разработке: как делать интернациональные проекты и говорить с пользователями на одном языке
  </span>
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 
Учимся создавать пакеты Python
<h2 class="tm-article-snip

## Задание

А что если мы хотим зайти еще глубже по дереву тегов и, например, для каждого заголовка поста найти никнейм юзера, который написал этот пост, и время написания поста?

Для этого надо снова зайти в просмотор кода страницы и увидеть, что там просиходит что-то такое:

(Заодно обратите внимание, как пишутся комменты в html)

In [47]:
all_articles = soup.find_all('article')
print(type(all_articles))
print(len(all_articles))

<class 'bs4.element.ResultSet'>
20


In [61]:
all_articles[0].find('h2', {'class': 'tm-article-snippet__title_h2'}).text

'Сборка мусора в неисправных JVM, проактивный подход'

In [62]:
all_articles[0].find('span', {'class': 'tm-user-info'}).text.strip()

'Shyhartskoi'

In [63]:
all_articles[0].find('span', {'class': 'tm-article-snippet__datetime-published'}).text.strip()

'сегодня в 17:00'

In [74]:
all_articles[0].find('span', {'class': 'tm-article-snippet__datetime-published'}).text 
# Не подходит, хотелось бы нормальную дату иметь под рукой

'сегодня в 17:00'

In [79]:
print(all_articles[0].find('span', {'class': 'tm-article-snippet__datetime-published'}).prettify())

<span class="tm-article-snippet__datetime-published">
 <time datetime="2022-11-18T14:00:01.000Z" title="2022-11-18, 17:00">
  сегодня в 17:00
 </time>
</span>


In [81]:
all_articles[0].find('span', {'class': 'tm-article-snippet__datetime-published'}).time['title']

'2022-11-18, 17:00'

In [85]:
import pandas as pd

In [86]:
def process_single_arcticle(bs4_article):
    return_dict = {
        'title': bs4_article.find('h2', {'class': 'tm-article-snippet__title_h2'}).text,
        'username': bs4_article.find('span', {'class': 'tm-user-info'}).text.strip(), 
        'create_date': bs4_article.find('span', {'class': 'tm-article-snippet__datetime-published'}).time['title']
    }
    return return_dict
        
def process_all_arcticles(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    all_articles = soup.find_all('article')
    rows = [process_single_arcticle(art) for art in all_articles]
    df = pd.DataFrame(rows, columns=['title', 'username', 'create_date'])
    return df

In [87]:
process_all_arcticles(response.text)

,title,username,create_date
0,"Сборка мусора в неисправных JVM, проактивный п...",Shyhartskoi,"2022-11-18, 17:00"
1,Трудности перевода в разработке: как делать ин...,vileven,"2022-11-18, 16:53"
2,Учимся создавать пакеты Python,ph_piter,"2022-11-18, 16:46"
3,ТОП-10 онлайн-ресурсов для прокачки навыков фр...,lena_ryan,"2022-11-18, 16:26"
4,Отечественная образовательная микроэлектроника...,akaz_zaka,"2022-11-18, 16:19"
5,Можно ли полноценно заменить VMware vSphere?,MSIVANOV,"2022-11-18, 16:10"
6,Почему тяжело писать про хороший код?,DyadichenkoGA,"2022-11-18, 16:04"
7,Основные архитектурные шаблоны построения ПО,Bright_Translate,"2022-11-18, 16:00"
8,Оформляем приложения по ГОСТ 7.32 в MS Word и ...,a1exjob,"2022-11-18, 15:59"
9,Как нанять Java-разработчика всего за один ден...,StanYurk,"2022-11-18, 15:54"


## Хорошие статьи по теме:

https://realpython.com/python-requests/   
[https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/](https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/)
